# UIUC Reboot Capstone Project

## Overview
You work for a regional soft drink distribution center that has access to invoice data from convenience stores (invoices.csv), product data (items.csv), and customer reviews (the files in reviews/).

You would like to examine the data for a few features, then plug them into a recommendation engine to see if you can improve sales.

### Loading Data

In [ ]:
import pandas as pd
import glob as glob
import zipfile
import numpy as np               # to get unique values from list using numpy.unique 


In [3]:
# Basic dataframe info